In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import pickle

In [2]:
import ipynb_path
now_file_name = ipynb_path.get().split('/')[-1].replace('.ipynb','')

input_path = '../input/'
status_file_name = 'status.csv'
station_file_name = 'station.csv'
trip_file_name = 'trip.csv'
weather_file_name = 'weather.csv'
output_path = '../output/'
model_path = '../model/'

In [3]:
#実行したらコメントアウトする
#保存ファイルの重複が起こる可能性がある
###########################################################
# もう一度notebookを動かす際はモデルリストのstart_numとend_numを
# 設定してください
###########################################################
st_in = input('初めの数字を入力してください')
end_in = input('終わりの数字を入力してください')
start_num = int(st_in)
end_num = int(end_in)

model_name_list = [f'model_{i}_{now_file_name}.sav' for i in range(start_num, end_num)]
submit_file_name_list = [f'submission_{i}_{now_file_name}.csv' for i in range(start_num,end_num)]
model_submit_dict = dict(zip(model_name_list,submit_file_name_list))

初めの数字を入力してください 0
終わりの数字を入力してください 20


In [4]:
status = pd.read_csv(input_path + status_file_name)
station = pd.read_csv(input_path + station_file_name)
weather = pd.read_csv(input_path + weather_file_name)


#statusのyear, month, dayを結合してdatetime型に
status['date'] = status['year'].astype(str) + '/' + status['month'].astype(str).str.zfill(2).astype(str) + '/' + status['day'].astype(str).str.zfill(2).astype(str)
status['date'] = pd.to_datetime(status['date'])

weather['date'] = pd.to_datetime(weather['date'])

train_sta_wea = pd.merge(status,weather, on = 'date', how ='left')

train_sta_wea['events'] = train_sta_wea['events'].fillna('なし')

le = preprocessing.LabelEncoder()
train_sta_wea['events'] = le.fit_transform(train_sta_wea['events'].values.tolist())
train_sta_wea = pd.merge(train_sta_wea, station[['station_id', 'city']], how = 'left')

train_sta_wea_pre_1 = train_sta_wea[train_sta_wea['predict'] == 1]

train_sta_wea_pre_0 = train_sta_wea[train_sta_wea['predict'] == 0]

#曜日を追加
train_sta_wea['date']=pd.to_datetime(train_bsta_wea['date'])
train_sta_wea['weekday']=train_sta_wea['date'].dt.weekday

In [17]:
#重回帰分析
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

class make_tr_va_te():
    def __init__(self, df, train_end_next_date):
        self.df = df
        self.train_end_next_date = train_end_next_date
        
    def make_train_data(self):
        train_all = self.df[self.df['predict'] == 0]
        train = train_all[train_all['date'] < self.train_end_next_date]
        train_notna = train[train['bikes_available'].notna()]
        return train_notna
        
    def make_valid_data(self):
        valid_all = self.df[self.df['predict'] == 0]
        valid = valid_all[(self.train_end_next_date <= valid_all['date']) & (valid_all['date'] < (self.train_end_next_date + relativedelta(months = 1)))]
        valid_notna = valid[valid['bikes_available'].notna()]
        return valid_notna
                                      
    def make_test_data(self):
        test_all = self.df[self.df['predict'] == 1]
        test = test_all[((self.train_end_next_date + relativedelta(months = 1)) <= test_all['date']) & (test_all['date']< (self.train_end_next_date + relativedelta(months = 2)))]
        return test
    
    def model_for_data(self, train, valid):      
        tr_X = train.drop(['id','predict','bikes_available','city','date'],axis=1)
        tr_y = train['bikes_available']
        va_X = valid.drop(['id','predict','bikes_available','city','date'],axis=1)
        va_y = valid['bikes_available']   
        return tr_X, tr_y, va_X, va_y
    
    def predict_for_data(self, test):
        te_X = test.drop(['id','predict','bikes_available','city','date'],axis=1)       
        return te_X
    
    def make_fit_model(self, tr_X, tr_y, va_X, va_y ):
        model = LinearRegression()
        model.fit(tr_X, tr_y)
        return model
    
    def model_and_valid_score(self):
        model = self.make_fit_model()
        valid_best_score = model.best_score['valid_1']['rmse']
        
        return model, valid_best_score
    
    def predict(self, model_file_name):
        import pickle
        train = self.make_train_data()
        valid = self.make_valid_data()
        test = self.make_test_data()
        tr_X, tr_y, va_X, va_y = self.model_for_data(train, valid)
        te_X = self.predict_for_data(test)
        model   = self.make_fit_model(tr_X, tr_y, va_X, va_y )
        pickle.dump(model, open(model_path + model_file_name, 'wb')) 
        va_pred = model.predict(va_X)
        y_pred = model.predict(te_X)
        rmse_score = np.sqrt(mse(va_y,va_pred))
        sub_index = test['id']
        sub_df = pd.DataFrame(list(zip(sub_index, y_pred)))
        print('*****')
        print(rmse_score)
        return sub_df, rmse_score
    



In [18]:
def month_range(start, stop, step = relativedelta(months = 1)):
    current = start
    while current < stop:
        yield current
        current += step

In [19]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

*****
3.9265042553556793
*****
4.0599377697046455
*****
4.319565186488821
*****
4.185624102513826
*****
4.450443588420891
*****
4.074178273797344
*****
4.302625917910384
*****
4.5189826472098416
*****
4.40707953796683
*****
4.302126506634736
*****
4.065503889854076
*****
3.94933292397741
***
***
CV score is 4.213492049986207
model_name is model_0_No_4_my.sav


'model_0_No_4_my.sav'

In [20]:
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')

***
***
CV score is 4.213492049986207
model_name is model_0_No_4_my.sav


In [21]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_0_No_4_my.csv


In [32]:
#KNeighborsRegressor
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as mse

class make_tr_va_te():
    def __init__(self, df, train_end_next_date):
        self.df = df
        self.train_end_next_date = train_end_next_date
        
    def make_train_data(self):
        train_all = self.df[self.df['predict'] == 0]
        train = train_all[train_all['date'] < self.train_end_next_date]
        train_notna = train[train['bikes_available'].notna()]
        return train_notna
        
    def make_valid_data(self):
        valid_all = self.df[self.df['predict'] == 0]
        valid = valid_all[(self.train_end_next_date <= valid_all['date']) & (valid_all['date'] < (self.train_end_next_date + relativedelta(months = 1)))]
        valid_notna = valid[valid['bikes_available'].notna()]
        return valid_notna
                                      
    def make_test_data(self):
        test_all = self.df[self.df['predict'] == 1]
        test = test_all[((self.train_end_next_date + relativedelta(months = 1)) <= test_all['date']) & (test_all['date']< (self.train_end_next_date + relativedelta(months = 2)))]
        return test
    
    def model_for_data(self, train, valid):      
        tr_X = train.drop(['id','predict','bikes_available','city','date'],axis=1)
        tr_y = train['bikes_available']
        va_X = valid.drop(['id','predict','bikes_available','city','date'],axis=1)
        va_y = valid['bikes_available']   
        return tr_X, tr_y, va_X, va_y
    
    def predict_for_data(self, test):
        te_X = test.drop(['id','predict','bikes_available','city','date'],axis=1)       
        return te_X
    
    def make_fit_model(self, tr_X, tr_y, va_X, va_y ):
        model = KNeighborsRegressor()
        model.fit(tr_X, tr_y)
        return model
    
    def model_and_valid_score(self):
        model = self.make_fit_model()
        valid_best_score = model.best_score['valid_1']['rmse']
        
        return model, valid_best_score
    
    def predict(self, model_file_name):
        import pickle
        train = self.make_train_data()
        valid = self.make_valid_data()
        test = self.make_test_data()
        tr_X, tr_y, va_X, va_y = self.model_for_data(train, valid)
        te_X = self.predict_for_data(test)
        model   = self.make_fit_model(tr_X, tr_y, va_X, va_y )
        pickle.dump(model, open(model_path + model_file_name, 'wb')) 
        va_pred = model.predict(va_X)
        y_pred = model.predict(te_X)
        rmse_score = np.sqrt(mse(va_y,va_pred))
        sub_index = test['id']
        sub_df = pd.DataFrame(list(zip(sub_index, y_pred)))
        print('*****')
        print(rmse_score)
        return sub_df, rmse_score
    




In [33]:
def month_range(start, stop, step = relativedelta(months = 1)):
    current = start
    while current < stop:
        yield current
        current += step

In [34]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

KeyboardInterrupt: 

In [ ]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

In [35]:
#汎用可能にする
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as mse

class make_tr_va_te():
    def __init__(self, df, train_end_next_date):
        self.df = df
        self.train_end_next_date = train_end_next_date
        
    def make_train_data(self):
        train_all = self.df[self.df['predict'] == 0]
        train = train_all[train_all['date'] < self.train_end_next_date]
        train_notna = train[train['bikes_available'].notna()]
        return train_notna
        
    def make_valid_data(self):
        valid_all = self.df[self.df['predict'] == 0]
        valid = valid_all[(self.train_end_next_date <= valid_all['date']) & (valid_all['date'] < (self.train_end_next_date + relativedelta(months = 1)))]
        valid_notna = valid[valid['bikes_available'].notna()]
        return valid_notna
                                      
    def make_test_data(self):
        test_all = self.df[self.df['predict'] == 1]
        test = test_all[((self.train_end_next_date + relativedelta(months = 1)) <= test_all['date']) & (test_all['date']< (self.train_end_next_date + relativedelta(months = 2)))]
        return test
    
    def model_for_data(self, train, valid):      
        tr_X = train.drop(['id','predict','bikes_available','city','date'],axis=1)
        tr_y = train['bikes_available']
        va_X = valid.drop(['id','predict','bikes_available','city','date'],axis=1)
        va_y = valid['bikes_available']   
        return tr_X, tr_y, va_X, va_y
    
    def predict_for_data(self, test):
        te_X = test.drop(['id','predict','bikes_available','city','date'],axis=1)       
        return te_X
    
    def make_fit_model(self, tr_X, tr_y, va_X, va_y ):
        model = KNeighborsRegressor()
        model.fit(tr_X, tr_y)
        return model
    
    def model_and_valid_score(self):
        model = self.make_fit_model()
        valid_best_score = model.best_score['valid_1']['rmse']
        
        return model, valid_best_score
    
    def predict(self, model, model_file_name):
        import pickle
        train = self.make_train_data()
        valid = self.make_valid_data()
        test = self.make_test_data()
        tr_X, tr_y, va_X, va_y = self.model_for_data(train, valid)
        te_X = self.predict_for_data(test)
        pickle.dump(model, open(model_path + model_file_name, 'wb')) 
        va_pred = model.predict(va_X)
        y_pred = model.predict(te_X)
        rmse_score = np.sqrt(mse(va_y,va_pred))
        sub_index = test['id']
        sub_df = pd.DataFrame(list(zip(sub_index, y_pred)))
        print('*****')
        print(rmse_score)
        return sub_df, rmse_score
    





In [36]:
#ラッソ回帰
from sklearn.linear_model import Lasso
def model_lasso(tr_X, tr_y):
    model = Lasso()
    model.fit(tr_X,tr_y)
    return model

In [38]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_lasso(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

*****
3.9315505671056528
*****
4.069846308212315
*****
4.331067409172134
*****
4.193090083497095
*****
4.44804331667126
*****
4.102317148986719
*****
4.323692564586363
*****
4.556095443206636
*****
4.42227588284393
*****
4.345449513107519
*****
4.092517202120235
*****
3.9422530954742605
***
***
CV score is 4.229849877915343
model_name is model_1_No_4_my.sav


'model_1_No_4_my.sav'

In [42]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_2_No_4_my.csv


In [39]:
#リッジ回帰
from sklearn.linear_model import Ridge
def model_Ridge(tr_X, tr_y):
    model = Ridge()
    model.fit(tr_X,tr_y)
    return model

In [40]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_Ridge(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

*****
3.926489606947237
*****
4.059935481033495
*****
4.319568902943038
*****
4.185619696258506
*****
4.450431349019066
*****
4.074193800028289
*****
4.302626242272293
*****
4.518991466030281
*****
4.407061215308665
*****
4.302126856261457
*****
4.065493795813253
*****
3.9493074171232245
***
***
CV score is 4.2134871524199005
model_name is model_2_No_4_my.sav


'model_2_No_4_my.sav'

In [41]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_2_No_4_my.csv


In [44]:
#エラスティックネット回帰
from sklearn.linear_model import ElasticNet
def model_ElasticNet(tr_X, tr_y):
    model = ElasticNet(alpha = 0.5)
    model.fit(tr_X,tr_y)
    return model

In [45]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_ElasticNet(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

*****
3.924977392584341
*****
4.063766212123566
*****
4.324165392645536
*****
4.190448437003381
*****
4.448839286789556
*****
4.109871421478162
*****
4.32841625127012
*****
4.5544400630224615
*****
4.422248031631271
*****
4.341653159801162
*****
4.083021464237497
*****
3.9350307119710273
***
***
CV score is 4.227239818713173
model_name is model_3_No_4_my.sav


'model_3_No_4_my.sav'

In [46]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_3_No_4_my.csv


In [48]:
#決定木
from sklearn.tree import DecisionTreeRegressor
def model_DecisionTreeRegressor(tr_X, tr_y):
    model = DecisionTreeRegressor(random_state=0)
    model.fit(tr_X,tr_y)
    return model

In [49]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_DecisionTreeRegressor(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

*****
4.326165346495986
*****
4.462414464966567
*****
4.922056718056632
*****
4.788951570318616
*****
4.8938470870863044
*****
4.591816177519901
*****
4.621631385339154
*****
4.67887449368253
*****
4.554896893924791
*****
4.68522737748424
*****
4.826687713713649
*****
4.815273752744345
***
***
CV score is 4.680653581777726
model_name is model_4_No_4_my.sav


'model_4_No_4_my.sav'

In [50]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_4_No_4_my.csv


In [51]:
#LinearSVR
from sklearn.svm import LinearSVR
def model_LinearSVR(tr_X, tr_y):
    model = LinearSVR(random_state=0)
    model.fit(tr_X,tr_y)
    return model

In [52]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_LinearSVR(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


*****
3.9284594345637167


/Users/ken/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


*****
5.298943795723864


/Users/ken/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


*****
5.360107381027779


/Users/ken/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


*****
8.299076547664155
***
***
CV score is 5.096147084391868
model_name is model_5_No_4_my.sav


'model_5_No_4_my.sav'

In [ ]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

In [ ]:
#決定木
from sklearn.tree import DecisionTreeRegressor
def model_DecisionTreeRegressor(tr_X, tr_y):
    model = DecisionTreeRegressor(random_state=0)
    model.fit(tr_X,tr_y)
    return model

In [ ]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_DecisionTreeRegressor(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

In [ ]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

In [ ]:
#決定木
from sklearn.tree import DecisionTreeRegressor
def model_DecisionTreeRegressor(tr_X, tr_y):
    model = DecisionTreeRegressor(random_state=0)
    model.fit(tr_X,tr_y)
    return model

In [ ]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = model_DecisionTreeRegressor(tr_X, tr_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model,model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

In [ ]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

In [2]:
import pandas as pd
e = pd.read_csv('../output/submission_1_No_4_my.csv')

In [3]:
e.shape

(193199, 2)

In [4]:
e.head()

,8761,7.7221345203172405
0,8762,7.712608
1,8763,7.703080
2,8764,7.693553
3,8765,7.684026
4,8766,7.674499
